# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: fetch latest code</h2>
            <span style="color:#f71;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use GPT-4o and Claude-3.5-Sonnet, which are the slightly higher priced models. The costs are still low, but if you'd prefer to keep costs ultra low, please make the suggested switches to the models (3 cells down from here).
            </span>
        </td>
    </tr>
</table>

In [13]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [31]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [34]:
# initialize
# NOTE - option to use ultra-low cost models by uncommenting last 2 lines

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-haiku-4-5-20251001"

# Want to keep costs ultra-low? Uncomment these lines:
# OPENAI_MODEL = "gpt-4o-mini"
# CLAUDE_MODEL = "claude-3-haiku-20240307"

In [16]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [17]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [18]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [19]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [20]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [21]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [22]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [23]:
exec(pi)

Result: 3.141592658589
Execution Time: 6.893879 seconds


In [24]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

// Function for the calculation
double calculate(long iterations, int param1, int param2) {
    double result = 1.0;
    for (long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1 / j);
        j = i * param1 + param2;
        result += (1 / j);
    }
    return result;
}

int main() {
    using namespace std::chrono;
    auto start_time = high_resolution_clock::now();

    double result = calculate(100000000, 4, 1) * 4;

    auto end_time = high_resolution_clock::now();
    duration<double> exec_time = end_time - start_time;

    // Printing result with 12 decimal places
    std::cout << "Result: " << std::fixed << std::setprecision(12) << result << std::endl;
    std::cout << "Execution Time: " << exec_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [25]:
exec(pi)

Result: 3.141592658589
Execution Time: 6.354336 seconds


# Compiling C++ and executing

This next cell contains the command to compile a C++ file on my M1 Mac.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

In the next lab (day4), a student has contributed a full solution that compiles to efficient code on Mac, PC and Linux!

You can wait for this, or you can google (or ask ChatGPT!) for how to do this on your platform, then replace the lines below.
If you're not comfortable with this step, you can skip it for sure - I'll show you exactly how it performs on my Mac.


OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [26]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.136320792000 seconds


In [35]:
optimize_claude(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, long long param1, long long param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        long long j1 = i * param1 - param2;
        result -= (1.0 / j1);
        long long j2 = i * param1 + param2;
        result += (1.0 / j2);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100'000'000LL, 4LL, 1LL) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();
    
    std::chrono::duration<double> elapsed = end_time - start_time;
    
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << std::fixed << std::setprecision(6);
    std::cout << "Execution Time: " << elapsed.count() << " seconds" << std::endl;
    
    return 0;
}
```

In [36]:
# Repeat for Claude - again, use the right approach for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.126163 seconds


In [37]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [38]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 17.521985 seconds


In [39]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <cstdint>
#include <limits>
#include <ctime>

// Linear Congruential Generator
class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint64_t m = uint64_t(1) << 32)
        : value(seed), a(a), c(c), m(m) {}

    uint32_t next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    uint32_t value;
    const uint32_t a, c;
    const uint64_t m;
};

// Maximum subarray sum with Kadane’s algorithm
int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg_gen(seed);
    std::vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg_gen.next() % (max_val - min_val + 1) + min_val;
    }

    int64_t max_sum = std::numeric_limits<int64_t>::min();
    int64_t current_sum = 0;
    
    for (int i = 0; i < n; ++i) {
        current_sum = std::max(static_cast<int64_t>(random_numbers[i]), current_sum + random_number

In [41]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

optimized.cpp:67:59: error: no member named 'setprecision' in namespace 'std'
   67 |     std::cout << "Execution Time: " << std::fixed << std::setprecision(6)
      |                                                      ~~~~~^
1 error generated.
Result: 3.141592658589
Execution Time: 0.105210 seconds


In [42]:
optimize_claude(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <iomanip>
#include <limits>

class LCG {
private:
    uint32_t value;
    static constexpr uint32_t a = 1664525;
    static constexpr uint32_t c = 1013904223;
    
public:
    LCG(uint32_t seed) : value(seed) {}
    
    uint32_t next() {
        value = (static_cast<uint64_t>(a) * value + c) % (1ULL << 32);
        return value;
    }
};

long long max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg_gen(seed);
    std::vector<int> random_numbers;
    random_numbers.reserve(n);
    
    int range = max_val - min_val + 1;
    for (int i = 0; i < n; ++i) {
        uint32_t rand_val = lcg_gen.next() % range + min_val;
        random_numbers.push_back(rand_val);
    }
    
    long long max_sum = std::numeric_limits<long long>::min();
    for (int i = 0; i < n; ++i) {
        long long current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
       

In [43]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.530326 seconds


In [44]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [45]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [46]:
def optimize(python, model):
    if model=="GPT-4o" or model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude-3.5-Sonnet" or model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError(f"Unknown model: {model}. Available models: GPT-4o, Claude-3.5-Sonnet")
    for stream_so_far in result:
        yield stream_so_far        

In [47]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [48]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [49]:
# You'll need to change the code in the try block to compile the C++ code for your platform
# I pasted this into Claude's chat UI with a request for it to give me a version for an Intel PC,
# and it responded with something that looks perfect - you can try a similar approach for your platform.

# M1 Mac version to compile and execute optimized C++ code:

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [50]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [51]:
# Enhanced Gradio Interface with better design and functionality
enhanced_css = """
.gradio-container {
    max-width: 1400px !important;
    margin: 0 auto !important;
}

.header {
    text-align: center;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 20px;
    border-radius: 10px;
    margin-bottom: 20px;
}

.code-section {
    background: #f8f9fa;
    border-radius: 8px;
    padding: 15px;
    margin: 10px 0;
}

.python-code {
    background-color: #306998 !important;
    color: white !important;
    border: 2px solid #1e3a8a !important;
}

.cpp-code {
    background-color: #00599C !important;
    color: white !important;
    border: 2px solid #003d82 !important;
}

.python-output {
    background-color: #f0f8ff !important;
    border: 2px solid #306998 !important;
    color: #1e3a8a !important;
}

.cpp-output {
    background-color: #f0f8ff !important;
    border: 2px solid #00599C !important;
    color: #003d82 !important;
}

.performance-metrics {
    background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%);
    border-radius: 8px;
    padding: 15px;
    margin: 10px 0;
    text-align: center;
}

.button-row {
    display: flex;
    gap: 10px;
    justify-content: center;
    margin: 15px 0;
}

.convert-btn {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    color: white !important;
    border: none !important;
    padding: 12px 24px !important;
    border-radius: 6px !important;
    font-weight: bold !important;
}

.run-btn {
    background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%) !important;
    color: white !important;
    border: none !important;
    padding: 10px 20px !important;
    border-radius: 6px !important;
}

.model-selector {
    background: white !important;
    border: 2px solid #667eea !important;
    border-radius: 6px !important;
}
"""

with gr.Blocks(css=enhanced_css, title="Python to C++ Code Optimizer", theme=gr.themes.Soft()) as ui:
    
    # Header Section
    with gr.Row():
        gr.HTML("""
        <div class="header">
            <h1>🚀 Python to C++ Code Optimizer</h1>
            <p>Convert your Python code to high-performance C++ using AI models</p>
        </div>
        """)
    
    # Main Content Area
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Python Code Input")
            python = gr.Textbox(
                label="Python Code:", 
                value=python_hard, 
                lines=15,
                placeholder="Enter your Python code here...",
                elem_classes=["python-code"],
                show_copy_button=True
            )
            
            with gr.Row():
                gr.Markdown("### 🤖 AI Model Selection")
                model = gr.Dropdown(
                    ["GPT-4o", "Claude-3.5-Sonnet"], 
                    label="Select AI Model", 
                    value="GPT-4o",
                    elem_classes=["model-selector"]
                )
            
            with gr.Row(elem_classes=["button-row"]):
                convert = gr.Button("🔄 Convert to C++", elem_classes=["convert-btn"])
        
        with gr.Column(scale=1):
            gr.Markdown("### ⚡ Generated C++ Code")
            cpp = gr.Textbox(
                label="C++ Code:", 
                lines=15,
                placeholder="Generated C++ code will appear here...",
                elem_classes=["cpp-code"],
                show_copy_button=True
            )
    
    # Execution Section
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🐍 Python Execution")
            with gr.Row(elem_classes=["button-row"]):
                python_run = gr.Button("▶️ Run Python", elem_classes=["run-btn"])
            python_out = gr.TextArea(
                label="Python Output:", 
                lines=8,
                elem_classes=["python-output"],
                placeholder="Python execution results will appear here..."
            )
        
        with gr.Column(scale=1):
            gr.Markdown("### 🚀 C++ Execution")
            with gr.Row(elem_classes=["button-row"]):
                cpp_run = gr.Button("▶️ Run C++", elem_classes=["run-btn"])
            cpp_out = gr.TextArea(
                label="C++ Output:", 
                lines=8,
                elem_classes=["cpp-output"],
                placeholder="C++ execution results will appear here..."
            )
    
    # Performance Metrics Section
    with gr.Row():
        gr.HTML("""
        <div class="performance-metrics">
            <h3>📊 Performance Comparison</h3>
            <p>Compare execution times and performance metrics between Python and C++ implementations</p>
        </div>
        """)
    
    # Event Handlers
    convert.click(
        optimize, 
        inputs=[python, model], 
        outputs=[cpp],
        show_progress=True
    )
    
    python_run.click(
        execute_python, 
        inputs=[python], 
        outputs=[python_out],
        show_progress=True
    )
    
    cpp_run.click(
        execute_cpp, 
        inputs=[cpp], 
        outputs=[cpp_out],
        show_progress=True
    )

# Launch with enhanced settings - using dynamic port to avoid conflicts
import socket

def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.listen(1)
        port = s.getsockname()[1]
    return port

# Find a free port automatically
free_port = find_free_port()
print(f"Launching Gradio interface on port: {free_port}")

ui.launch(
    inbrowser=True,
    share=False,
    server_name="127.0.0.1",
    server_port=free_port,
    show_error=True,
    quiet=False
)

Launching Gradio interface on port: 59372
* Running on local URL:  http://127.0.0.1:59372

To create a public link, set `share=True` in `launch()`.


In [ ]:
# Modern ChatGPT/Perplexity-style Interface with Full Functionality
import gradio as gr
import time
from datetime import datetime

# Modern ChatGPT/Perplexity-inspired CSS
modern_css = """
/* Modern ChatGPT/Perplexity Style Interface */
.gradio-container {
    max-width: 1400px !important;
    margin: 0 auto !important;
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
}

.modern-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 24px;
    border-radius: 16px;
    margin-bottom: 24px;
    text-align: center;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
}

.modern-header h1 {
    margin: 0;
    font-size: 32px;
    font-weight: 700;
    letter-spacing: -0.5px;
}

.modern-header p {
    margin: 12px 0 0 0;
    opacity: 0.9;
    font-size: 18px;
    font-weight: 400;
}

.code-section {
    background: #ffffff;
    border-radius: 16px;
    padding: 24px;
    margin: 20px 0;
    border: 1px solid #e2e8f0;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.05);
}

.python-input {
    background: #f8fafc !important;
    border: 2px solid #e2e8f0 !important;
    border-radius: 12px !important;
    padding: 16px !important;
    font-family: 'Monaco', 'Menlo', 'Ubuntu Mono', monospace !important;
    font-size: 14px !important;
    color: #1e293b !important;
    line-height: 1.5 !important;
}

.python-input:focus {
    border-color: #3b82f6 !important;
    box-shadow: 0 0 0 3px rgba(59, 130, 246, 0.1) !important;
}

.cpp-output {
    background: #f1f5f9 !important;
    border: 2px solid #cbd5e1 !important;
    border-radius: 12px !important;
    padding: 16px !important;
    font-family: 'Monaco', 'Menlo', 'Ubuntu Mono', monospace !important;
    font-size: 14px !important;
    color: #0f172a !important;
    line-height: 1.5 !important;
}

.model-selector {
    background: white !important;
    border: 2px solid #e2e8f0 !important;
    border-radius: 12px !important;
    padding: 12px 16px !important;
    font-size: 16px !important;
    color: #374151 !important;
}

.model-selector:focus {
    border-color: #3b82f6 !important;
    box-shadow: 0 0 0 3px rgba(59, 130, 246, 0.1) !important;
}

.modern-button {
    background: linear-gradient(135deg, #3b82f6 0%, #1d4ed8 100%) !important;
    color: white !important;
    border: none !important;
    border-radius: 12px !important;
    padding: 14px 28px !important;
    font-weight: 600 !important;
    font-size: 16px !important;
    cursor: pointer !important;
    transition: all 0.2s ease !important;
    box-shadow: 0 4px 6px rgba(59, 130, 246, 0.2) !important;
}

.modern-button:hover {
    transform: translateY(-2px) !important;
    box-shadow: 0 8px 12px rgba(59, 130, 246, 0.3) !important;
}

.run-button {
    background: linear-gradient(135deg, #10b981 0%, #059669 100%) !important;
    color: white !important;
    border: none !important;
    border-radius: 10px !important;
    padding: 12px 24px !important;
    font-weight: 600 !important;
    font-size: 14px !important;
    cursor: pointer !important;
    transition: all 0.2s ease !important;
    box-shadow: 0 4px 6px rgba(16, 185, 129, 0.2) !important;
}

.run-button:hover {
    transform: translateY(-1px) !important;
    box-shadow: 0 6px 8px rgba(16, 185, 129, 0.3) !important;
}

.output-section {
    background: #f8fafc;
    border: 1px solid #e2e8f0;
    border-radius: 12px;
    padding: 16px;
    margin: 12px 0;
    font-family: 'Monaco', 'Menlo', 'Ubuntu Mono', monospace;
    font-size: 13px;
    line-height: 1.4;
    color: #374151;
    min-height: 100px;
    overflow-y: auto;
}

.python-output {
    background: #fef3c7 !important;
    border: 2px solid #f59e0b !important;
    color: #92400e !important;
}

.cpp-output {
    background: #dbeafe !important;
    border: 2px solid #3b82f6 !important;
    color: #1e40af !important;
}

.performance-card {
    background: linear-gradient(135deg, #f0f9ff 0%, #e0f2fe 100%);
    border: 1px solid #0ea5e9;
    border-radius: 12px;
    padding: 20px;
    margin: 16px 0;
    text-align: center;
}

.performance-card h3 {
    margin: 0 0 12px 0;
    color: #0c4a6e;
    font-size: 18px;
    font-weight: 600;
}

.performance-metric {
    display: inline-block;
    background: white;
    border-radius: 8px;
    padding: 8px 16px;
    margin: 4px;
    font-weight: 600;
    color: #0c4a6e;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

.status-indicator {
    display: inline-block;
    width: 8px;
    height: 8px;
    border-radius: 50%;
    margin-right: 8px;
}

.status-ready { background: #10b981; }
.status-running { background: #f59e0b; }
.status-complete { background: #3b82f6; }
.status-error { background: #ef4444; }
"""

# Create the modern ChatGPT/Perplexity-style interface
with gr.Blocks(css=modern_css, title="Python to C++ Code Optimizer - Modern Style", theme=gr.themes.Soft()) as modern_ui:
    
    # Header Section
    gr.HTML("""
    <div class="modern-header">
        <h1>🚀 Python to C++ Code Optimizer</h1>
        <p>AI-powered code conversion with real-time execution and performance analysis</p>
    </div>
    """)
    
    # Main Content Area
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Python Code Input")
            python_input = gr.Textbox(
                label="Python Code:", 
                value=python_hard, 
                lines=15,
                placeholder="Enter your Python code here...",
                elem_classes=["python-input"],
                show_copy_button=True
            )
            
            with gr.Row():
                gr.Markdown("### 🤖 AI Model Selection")
                model_selector = gr.Dropdown(
                    ["GPT-4o", "Claude-3.5-Sonnet"], 
                    label="Select AI Model", 
                    value="GPT-4o",
                    elem_classes=["model-selector"]
                )
            
            with gr.Row():
                convert_btn = gr.Button("🔄 Convert to C++", elem_classes=["modern-button"])
        
        with gr.Column(scale=1):
            gr.Markdown("### ⚡ Generated C++ Code")
            cpp_output = gr.Textbox(
                label="C++ Code:", 
                lines=15,
                placeholder="Generated C++ code will appear here...",
                elem_classes=["cpp-output"],
                show_copy_button=True
            )
    
    # Execution Section
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🐍 Python Execution")
            with gr.Row():
                python_run_btn = gr.Button("▶️ Run Python", elem_classes=["run-button"])
            python_result = gr.TextArea(
                label="Python Output:", 
                lines=8,
                elem_classes=["output-section", "python-output"],
                placeholder="Python execution results will appear here..."
            )
        
        with gr.Column(scale=1):
            gr.Markdown("### 🚀 C++ Execution")
            with gr.Row():
                cpp_run_btn = gr.Button("▶️ Run C++", elem_classes=["run-button"])
            cpp_result = gr.TextArea(
                label="C++ Output:", 
                lines=8,
                elem_classes=["output-section", "cpp-output"],
                placeholder="C++ execution results will appear here..."
            )
    
    # Performance Metrics Section
    gr.HTML("""
    <div class="performance-card">
        <h3>📊 Performance Comparison</h3>
        <div>
            <span class="performance-metric">Python: ~6.3s</span>
            <span class="performance-metric">C++: ~0.6s</span>
            <span class="performance-metric">Speedup: 10x</span>
        </div>
        <p style="margin: 12px 0 0 0; color: #64748b; font-size: 14px;">
            Compare execution times and performance metrics between Python and C++ implementations
        </p>
    </div>
    """)
    
    # Event Handlers - Same functionality as original
    convert_btn.click(
        optimize, 
        inputs=[python_input, model_selector], 
        outputs=[cpp_output],
        show_progress=True
    )
    
    python_run_btn.click(
        execute_python, 
        inputs=[python_input], 
        outputs=[python_result],
        show_progress=True
    )
    
    cpp_run_btn.click(
        execute_cpp, 
        inputs=[cpp_output], 
        outputs=[cpp_result],
        show_progress=True
    )

# Launch with dynamic port selection
import socket

def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.listen(1)
        port = s.getsockname()[1]
    return port

# Find a free port automatically
free_port = find_free_port()
print(f"🚀 Launching Modern Python to C++ Code Optimizer on port: {free_port}")

modern_ui.launch(
    inbrowser=True,
    share=False,
    server_name="127.0.0.1",
    server_port=free_port,
    show_error=True,
    quiet=False
)


NameError: name 'python_hard' is not defined